In [1]:
#import relevant libraries
import pandas as pd
import os
import re

# CADEC Dataset

## Creating Datafarme 1 - store text and their ID

In [87]:
# assign directory
directory = '/Users/teonaristova/Desktop/Classes/NLP/Project/Datasets/CADEC/CSIRO_DAP_Metadata/cadec/text/'

txt_id = []
text = []
# iterate over files in the directory
for filename in os.listdir(directory):
    
    #store the filename as the id of the text
    file = filename.replace(".txt", "")
    txt_id.append(file)
    
    file_open = directory + filename
    #open the text file and store the text
    with open(file_open) as f:
        contents = f.read()
        
    text.append(contents)
    

#create a dataframe that holds the filename as the text id as one column
#and the content of the text file as another column
d = {'txt_id': txt_id, 'text': text}
text_df = pd.DataFrame(data=d)
text_df.head()

,txt_id,text
0,LIPITOR.86,"headaches, pain in throat , tingling in side o..."
1,LIPITOR.92,"Muscle aches and weakness in neck, arms, shoul..."
2,LIPITOR.952,I have taken Lipitor for 4 years with no probl...
3,LIPITOR.946,"Aches and pain from head to toe, very grouchy...."
4,LIPITOR.45,"Cannot be sure it is the drug, but around the ..."


In [88]:
#save as a csv file
text_df.to_csv(r'/Users/teonaristova/Desktop/Classes/NLP/Project/Standardized_Dataset/text_cadec.csv', index=False)

In [4]:
text_df[text_df['txt_id'] == 'LEXAPRO.1']

,txt_id,text


In [5]:
len(text_df)

1250

## Creating Dataset 2 - text ID, ADR lines, and ADR (long)

In [80]:
# assign directory
directory = '/Users/teonaristova/Desktop/Classes/NLP/Project/Datasets/CADEC/CSIRO_DAP_Metadata/cadec/original/'

count = 0
#loop through all the files
for filename in os.listdir(directory):
    
    count += 1
    file_open = directory + filename
    
    try:
        temp_df = pd.read_csv(file_open, sep='^([^\s]*)\s', engine='python', header=None).drop(0, axis=1)
    except:
        continue
    
    #add a column that would indicate the id
    file = filename.replace(".ann", "")
    #print(type(file))
    temp_df['txt_id'] = file

    #construct a dataframe with all the entries
    if count == 1:
        final_df = temp_df
    else:
        final_df = pd.concat([final_df, temp_df])

In [81]:
#separate some of the existing columns into new separated colums
final_df[['type', 'rest']] = final_df[2].str.split(' ', 1, expand=True)
final_df[['location', 'symptom']] = final_df['rest'].str.split('\t', 1, expand=True)
final_df[['start', 'end']] = final_df['location'].str.split(' ', 1, expand=True)

#keep only the ADR and anotator notes ones
final_df = final_df[(final_df['type'] == 'ADR')]
# keep only particular columns
final_df = final_df[[1,2,'type', 'symptom', 'start', 'end', 'txt_id']]

#rename columns
final_df = final_df.rename(columns={1: 'loc', 2: 'original_ann'})


/var/folders/82/01mnxn_n0356pz_t4d1yxs280000gn/T/ipykernel_33130/1818809276.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  final_df[['type', 'rest']] = final_df[2].str.split(' ', 1, expand=True)
/var/folders/82/01mnxn_n0356pz_t4d1yxs280000gn/T/ipykernel_33130/1818809276.py:3: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  final_df[['location', 'symptom']] = final_df['rest'].str.split('\t', 1, expand=True)
/var/folders/82/01mnxn_n0356pz_t4d1yxs280000gn/T/ipykernel_33130/1818809276.py:4: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  final_df[['start', 'end']] = final_df['location'].str.split(' ', 1, expand=True)


In [82]:
final_df.head(20)

,loc,original_ann,type,symptom,start,end,txt_id
0,T1,ADR 57 67\tleg cramps,ADR,leg cramps,57,67,LIPITOR.95
1,T2,ADR 158 164\tcramps,ADR,cramps,158,164,LIPITOR.95
0,T1,ADR 0 30\tSevere pain in my calf muscles,ADR,Severe pain in my calf muscles,0,30,LIPITOR.969
3,T4,ADR 332 351\tpain was too severe,ADR,pain was too severe,332,351,LIPITOR.969
4,T5,ADR 384 418\tcould not even walk in the morning,ADR,could not even walk in the morning,384,418,LIPITOR.969
5,T6,ADR 424 430\tcramps,ADR,cramps,424,430,LIPITOR.969
8,T9,ADR 577 581\tpain,ADR,pain,577,581,LIPITOR.969
0,T1,ADR 0 13\tconstant pain,ADR,constant pain,0,13,LIPITOR.81
1,T2,ADR 30 44\tabdominal pain,ADR,abdominal pain,30,44,LIPITOR.81
2,T3,ADR 46 63\tintermittent pain,ADR,intermittent pain,46,63,LIPITOR.81


In [83]:
#split the end column by spaces
final_df['end'] = final_df.apply(lambda x: str(x.end).split(" "), axis=1)
#keep only the last value of the list
final_df['end'] = final_df['end'].apply(lambda a: a[-1])
final_df['end'] = final_df['end'].astype(int)
#keep only the row with the max end if duplicates
final_df = final_df.groupby(['txt_id', 'start'], group_keys=False).apply(lambda x: x.loc[x.end.idxmax()]).reset_index(drop=True)

In [84]:
final_df.head(20)

,loc,original_ann,type,symptom,start,end,txt_id
0,T2,ADR 29 50\tlittle blurred vision,ADR,little blurred vision,29,50,ARTHROTEC.1
1,T8,ADR 437 453\tfeel a bit weird,ADR,feel a bit weird,437,453,ARTHROTEC.1
2,T4,ADR 62 78\tgastric problems,ADR,gastric problems,62,78,ARTHROTEC.1
3,T1,ADR 9 19\tbit drowsy,ADR,bit drowsy,9,19,ARTHROTEC.1
4,T3,ADR 0 12\tHunger pangs,ADR,Hunger pangs,0,12,ARTHROTEC.10
5,T3,ADR 107 119\tcanker sores,ADR,canker sores,107,119,ARTHROTEC.100
6,T4,ADR 133 141\theadache,ADR,headache,133,141,ARTHROTEC.100
7,T1,ADR 48 64\tvaginal bleeding,ADR,vaginal bleeding,48,64,ARTHROTEC.100
8,T2,ADR 93 105\tstomach pain,ADR,stomach pain,93,105,ARTHROTEC.100
9,T3,ADR 102 110\tcramping,ADR,cramping,102,110,ARTHROTEC.101


In [85]:
#save as a csv file
final_df.to_csv(r'/Users/teonaristova/Desktop/Classes/NLP/Project/Standardized_Dataset/labeled_cadec.csv', index=False)

In [193]:
#Example of the dataset
directory = '/Users/teonaristova/Desktop/Classes/NLP/Project/Datasets/CADEC/CSIRO_DAP_Metadata/cadec/sct/'
filename = 'ARTHROTEC.1.ann'
file_open = directory + filename
pd.read_csv(file_open, sep='^([^\s]*)\s', engine='python', header=None).drop(0, axis=1)

,1,2
0,TT1,271782001 | Drowsy | 9 19\tbit drowsy
1,TT2,246636008 | Blurred vision - hazy | 29 50\tlit...
2,TT4,162076009 | Excessive upper gastrointestinal g...
3,TT3,3384011000036100 | Arthrotec | 93 102\tArthrotec
4,TT5,3723001 | Arthritis | 179 188\tarthritis
5,TT6,102498003 | Agony | or 76948002|Severe pain| 2...
6,TT7,22253000 | Pain | 412 417\tpains
7,TT8,367391008 | Malaise | 437 453\tfeel a bit weird


# PsyTar Dataset

## Creating Datafarme 1 - store text and their ID

In [179]:
psytar = pd.read_excel(open('/Users/teonaristova/Desktop/Classes/NLP/Project/Datasets/PsyTAR_dataset.xlsx', 'rb'),sheet_name='Sample') 

In [180]:
#combine the side effect and comment as a single comment (this is how the sentence labeling is done)
psytar['text'] = psytar['side-effect'] + ' ' + psytar['comment']

#keep only the relevant columns which are the id of the text and the text itself
psytar = psytar[['drug_id', 'text']]

In [181]:
psytar.head()

,drug_id,text
0,lexapro.1,"extreme weight gain, short-term memory loss, h..."
1,lexapro.2,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...
2,lexapro.3,"Nausea, Blurred Vision, 3 to 5 hours sleep, Su..."
3,lexapro.4,"Plenty! First 10 days were HORRIBLE, like a lo..."
4,lexapro.5,"Chronic cough, weight gain, no sexual interest..."


In [182]:
len(psytar)

891

In [89]:
#save as a csv file
psytar.to_csv(r'/Users/teonaristova/Desktop/Classes/NLP/Project/Standardized_Dataset/text_psytar.csv', index=False)

## Creating Dataset 2 - text ID, ADR lines, and ADR (long)

In [160]:
psytar_adr = pd.read_excel(open('/Users/teonaristova/Desktop/Classes/NLP/Project/Datasets/PsyTAR_dataset.xlsx', 'rb'),sheet_name='ADR_Identified') 

In [161]:
psytar_adr["new_id"] = psytar_adr.index
#reshape DataFrame from wide format to long format
psytar_adr = pd.wide_to_long(psytar_adr, ["ADR"], i="new_id", j="number")

#drop all the NaN values for the ADR
psytar_adr = psytar_adr.dropna(subset=['ADR'])

In [162]:
mapping_id = pd.read_excel(open('/Users/teonaristova/Desktop/Classes/NLP/Project/Datasets/PsyTAR_dataset.xlsx', 'rb'),sheet_name='Sentence_Labeling')
#subset only the comment id and sentence id columns whcih we will use for mapping
mapping_id = mapping_id[['drug_id', 'comment_id']]

In [163]:
#map sentences with their corresponding comment ids in the psytar_adr
merged_df = psytar_adr.merge(mapping_id, on='drug_id', how='left')
merged_df = merged_df.drop_duplicates()

In [164]:
#merge back the comment/sentence based on the comment id
combined_df = merged_df.merge(psytar, on='drug_id', how='left')
combined_df

,sentence_index,sentences,drug_id,id,ADR,comment_id,text
0,1,"extreme weight gain, short-term memory loss, h...",lexapro.1,1.0,extreme weight gain,1.0,"extreme weight gain, short-term memory loss, h..."
1,1,COMPLETELY DESTROYED SEXUALLY FUNCTIONING .,lexapro.2,2.0,COMPLETELY DESTROYED SEXUALLY FUNCTIONING,2.0,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...
2,4,Just TWO tablets of Lexapro 10mg completely de...,lexapro.2,3.0,completely destroyed my sexual functioning,2.0,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...
3,5,It's called PSSD: post-SSRI sexual dysfunction.,lexapro.2,4.0,PSSD,2.0,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...
4,8,And there is a chance that it will give you PS...,lexapro.2,5.0,PSSD,2.0,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...
...,...,...,...,...,...,...,...
4810,1,All these happened the day of my first pill: L...,lexapro.140,443.0,disorientation,140.0,All these happened the day of my first pill: L...
4811,1,"sweating,dizziness, severe dry mouth (coughing...",effexorXR.109,1908.0,loss of balance,109.0,"sweating,dizziness, severe dry mouth (coughing..."
4812,1,All these happened the day of my first pill: L...,lexapro.140,443.0,Convulsions,140.0,All these happened the day of my first pill: L...
4813,1,"sweating,dizziness, severe dry mouth (coughing...",effexorXR.109,1908.0,complete loss of sex drive,109.0,"sweating,dizziness, severe dry mouth (coughing..."


In [165]:
combined_df['start'] = combined_df.apply(lambda x: str(x.text).find(str(x.ADR)), axis=1)
combined_df['length'] = combined_df.apply(lambda x: len(x.ADR), axis=1)
combined_df['end'] = combined_df['start'] + combined_df['length']
combined_df['ADRs'] = combined_df['ADR'].apply(str.lower)
combined_df

,sentence_index,sentences,drug_id,id,ADR,comment_id,text,start,length,end,ADRs
0,1,"extreme weight gain, short-term memory loss, h...",lexapro.1,1.0,extreme weight gain,1.0,"extreme weight gain, short-term memory loss, h...",0,19,19,extreme weight gain
1,1,COMPLETELY DESTROYED SEXUALLY FUNCTIONING .,lexapro.2,2.0,COMPLETELY DESTROYED SEXUALLY FUNCTIONING,2.0,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...,0,41,41,completely destroyed sexually functioning
2,4,Just TWO tablets of Lexapro 10mg completely de...,lexapro.2,3.0,completely destroyed my sexual functioning,2.0,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...,129,42,171,completely destroyed my sexual functioning
3,5,It's called PSSD: post-SSRI sexual dysfunction.,lexapro.2,4.0,PSSD,2.0,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...,205,4,209,pssd
4,8,And there is a chance that it will give you PS...,lexapro.2,5.0,PSSD,2.0,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...,205,4,209,pssd
...,...,...,...,...,...,...,...,...,...,...,...
4810,1,All these happened the day of my first pill: L...,lexapro.140,443.0,disorientation,140.0,All these happened the day of my first pill: L...,389,14,403,disorientation
4811,1,"sweating,dizziness, severe dry mouth (coughing...",effexorXR.109,1908.0,loss of balance,109.0,"sweating,dizziness, severe dry mouth (coughing...",519,15,534,loss of balance
4812,1,All these happened the day of my first pill: L...,lexapro.140,443.0,Convulsions,140.0,All these happened the day of my first pill: L...,405,11,416,convulsions
4813,1,"sweating,dizziness, severe dry mouth (coughing...",effexorXR.109,1908.0,complete loss of sex drive,109.0,"sweating,dizziness, severe dry mouth (coughing...",536,26,562,complete loss of sex drive


In [166]:
#now we need to merge with the ADR based on a lexicon

#first let's load the data that contains this information
lexicon_adr = mapping_id = pd.read_excel(open('/Users/teonaristova/Desktop/Classes/NLP/Project/Datasets/PsyTAR_dataset.xlsx', 'rb'),sheet_name='ADR_Mapped')
lexicon_adr = lexicon_adr[['drug_id', 'sentence_index', 'ADRs', 'UMLS1']]
lexicon_adr.head()

,drug_id,sentence_index,ADRs,UMLS1
0,lexapro.1,1,extreme weight gain,C0000765 / Excessive body weight gain / Finding
1,lexapro.1,1,short-term memory loss,C0701811 / Poor short-term memory / Mental or ...
2,lexapro.1,1,hair loss,C0002170 / Alopecia / Disease or Syndrome
3,lexapro.2,1,completely destroyed sexually functioning,C0549622 / Sexual Dysfunction / Disease or Syn...
4,lexapro.2,4,completely destroyed my sexual functioning,C0549622 / Sexual Dysfunction / Disease or Syn...


In [167]:
len(lexicon_adr)

4813

In [171]:
#merge the dataset we created so far with the ones with the lexicon ADR values
final_df = combined_df.merge(lexicon_adr, on=['drug_id', 'sentence_index', 'ADRs'], how='left')
final_df

,sentence_index,sentences,drug_id,id,ADR,comment_id,text,start,length,end,ADRs,UMLS1
0,1,"extreme weight gain, short-term memory loss, h...",lexapro.1,1.0,extreme weight gain,1.0,"extreme weight gain, short-term memory loss, h...",0,19,19,extreme weight gain,C0000765 / Excessive body weight gain / Finding
1,1,COMPLETELY DESTROYED SEXUALLY FUNCTIONING .,lexapro.2,2.0,COMPLETELY DESTROYED SEXUALLY FUNCTIONING,2.0,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...,0,41,41,completely destroyed sexually functioning,C0549622 / Sexual Dysfunction / Disease or Syn...
2,4,Just TWO tablets of Lexapro 10mg completely de...,lexapro.2,3.0,completely destroyed my sexual functioning,2.0,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...,129,42,171,completely destroyed my sexual functioning,C0549622 / Sexual Dysfunction / Disease or Syn...
3,5,It's called PSSD: post-SSRI sexual dysfunction.,lexapro.2,4.0,PSSD,2.0,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...,205,4,209,pssd,C0549622 / Sexual Dysfunction / Disease or Syn...
4,8,And there is a chance that it will give you PS...,lexapro.2,5.0,PSSD,2.0,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...,205,4,209,pssd,C0549622 / Sexual Dysfunction / Disease or Syn...
...,...,...,...,...,...,...,...,...,...,...,...,...
4819,1,All these happened the day of my first pill: L...,lexapro.140,443.0,disorientation,140.0,All these happened the day of my first pill: L...,389,14,403,disorientation,C0233407 / Disorientation / Mental or Behavior...
4820,1,"sweating,dizziness, severe dry mouth (coughing...",effexorXR.109,1908.0,loss of balance,109.0,"sweating,dizziness, severe dry mouth (coughing...",519,15,534,loss of balance,C0241981 / Impairment of balance / Finding
4821,1,All these happened the day of my first pill: L...,lexapro.140,443.0,Convulsions,140.0,All these happened the day of my first pill: L...,405,11,416,convulsions,C4048158 / Convulsions / Sign or Symptom
4822,1,"sweating,dizziness, severe dry mouth (coughing...",effexorXR.109,1908.0,complete loss of sex drive,109.0,"sweating,dizziness, severe dry mouth (coughing...",536,26,562,complete loss of sex drive,C2981158 / Lack of libido / Finding


In [172]:
final_df[['UMLS_code', 'rest']]= final_df['UMLS1'].str.split('/', 1, expand=True)
final_df[['ann_symptom', 'type']]= final_df['rest'].str.split('/', 1, expand=True)
final_df.head()

/var/folders/82/01mnxn_n0356pz_t4d1yxs280000gn/T/ipykernel_75682/574311419.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  final_df[['UMLS_code', 'rest']]= final_df['UMLS1'].str.split('/', 1, expand=True)
/var/folders/82/01mnxn_n0356pz_t4d1yxs280000gn/T/ipykernel_75682/574311419.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  final_df[['ann_symptom', 'type']]= final_df['rest'].str.split('/', 1, expand=True)


,sentence_index,sentences,drug_id,id,ADR,comment_id,text,start,length,end,ADRs,UMLS1,UMLS_code,rest,ann_symptom,type
0,1,"extreme weight gain, short-term memory loss, h...",lexapro.1,1.0,extreme weight gain,1.0,"extreme weight gain, short-term memory loss, h...",0,19,19,extreme weight gain,C0000765 / Excessive body weight gain / Finding,C0000765,Excessive body weight gain / Finding,Excessive body weight gain,Finding
1,1,COMPLETELY DESTROYED SEXUALLY FUNCTIONING .,lexapro.2,2.0,COMPLETELY DESTROYED SEXUALLY FUNCTIONING,2.0,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...,0,41,41,completely destroyed sexually functioning,C0549622 / Sexual Dysfunction / Disease or Syn...,C0549622,Sexual Dysfunction / Disease or Syndrome,Sexual Dysfunction,Disease or Syndrome
2,4,Just TWO tablets of Lexapro 10mg completely de...,lexapro.2,3.0,completely destroyed my sexual functioning,2.0,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...,129,42,171,completely destroyed my sexual functioning,C0549622 / Sexual Dysfunction / Disease or Syn...,C0549622,Sexual Dysfunction / Disease or Syndrome,Sexual Dysfunction,Disease or Syndrome
3,5,It's called PSSD: post-SSRI sexual dysfunction.,lexapro.2,4.0,PSSD,2.0,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...,205,4,209,pssd,C0549622 / Sexual Dysfunction / Disease or Syn...,C0549622,Sexual Dysfunction / Disease or Syndrome,Sexual Dysfunction,Disease or Syndrome
4,8,And there is a chance that it will give you PS...,lexapro.2,5.0,PSSD,2.0,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...,205,4,209,pssd,C0549622 / Sexual Dysfunction / Disease or Syn...,C0549622,Sexual Dysfunction / Disease or Syndrome,Sexual Dysfunction,Disease or Syndrome


In [174]:
#subset only the relevant columns
final_df = final_df[['drug_id', 'text', 'ADR', 'start','end','UMLS_code', 'ann_symptom']]
final_df.head()

,drug_id,text,ADR,start,end,UMLS_code,ann_symptom
0,lexapro.1,"extreme weight gain, short-term memory loss, h...",extreme weight gain,0,19,C0000765,Excessive body weight gain
1,lexapro.2,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...,COMPLETELY DESTROYED SEXUALLY FUNCTIONING,0,41,C0549622,Sexual Dysfunction
2,lexapro.2,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...,completely destroyed my sexual functioning,129,171,C0549622,Sexual Dysfunction
3,lexapro.2,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...,PSSD,205,209,C0549622,Sexual Dysfunction
4,lexapro.2,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...,PSSD,205,209,C0549622,Sexual Dysfunction


In [177]:
#rename columns
final_df = final_df.rename(columns={'drug_id': 'txt_id', 'ADR': 'symptom'})
final_df.head()

,txt_id,text,symptom,start,end,UMLS_code,ann_symptom
0,lexapro.1,"extreme weight gain, short-term memory loss, h...",extreme weight gain,0,19,C0000765,Excessive body weight gain
1,lexapro.2,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...,COMPLETELY DESTROYED SEXUALLY FUNCTIONING,0,41,C0549622,Sexual Dysfunction
2,lexapro.2,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...,completely destroyed my sexual functioning,129,171,C0549622,Sexual Dysfunction
3,lexapro.2,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...,PSSD,205,209,C0549622,Sexual Dysfunction
4,lexapro.2,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...,PSSD,205,209,C0549622,Sexual Dysfunction


In [178]:
#save as a csv file
final_df.to_csv(r'/Users/teonaristova/Desktop/Classes/NLP/Project/Standardized_Dataset/labeled_psytar.csv', index=False)

# Annotated Drug Review Dataset

## Creating Dataset 2 - text ID, ADR lines, and ADR (long)

In [200]:
#open the text file
with open('/Users/teonaristova/Desktop/Classes/NLP/Project/Datasets/annotated_drug_reviews.txt') as f:
        contents = f.read()

In [201]:
contents

'headaches,||dizziness,||aching all over|!|headaches, dizziness, aching all over\nhair loss;||depression;||anger issues;||hostility;||visual acuity;||forgetfulness;||leg cramps|!|Substantial hair loss; depression; anger issues; hostility; diminished visual acuity; forgetfulness; minor leg cramps\nThinning hair,||extreme fatigue,||joint pain.||bone pain.||carpal tunnel syndrome||pain.||Insomnia||hot flashes|!|Thinning hair, extreme fatigue, awful joint and bone pain. Most distressing is the loss of the use of my hands. I have developed carpal tunnel syndrome in both hands and I am in alot of pain. Insomnia and hot flashes are also troublesome.\npain in right hip||pain in leg||neuropothy in feet.||trouble sleeping||neuropathy,|!|I had been on Tamoxofin for 5 years. I then took Evista for about 1 1 / 2 years when my doctor put me on Femara because it is more effective. I was fine in the beginning. After two years or so, I started having pain in my right hip and down my leg where it was un

In [202]:
file1 = open('/Users/teonaristova/Desktop/Classes/NLP/Project/Datasets/annotated_drug_reviews.txt', 'r')
Lines = file1.readlines()

drug_reviews = []
for line in Lines:
    drug_reviews.append(line)

In [212]:
review_dict = {}

for review in drug_reviews:
    
    #split the annotated symptoms from the text itself
    split_r = review.split('|!|')
    
    review_dict[split_r[1].replace('\n', '')] = split_r[0].split('||')

#we now get a dicionary where each key is a text and each value are list of symptoms
review_dict

{'headaches, dizziness, aching all over': ['headaches,',
  'dizziness,',
  'aching all over'],
 'Substantial hair loss; depression; anger issues; hostility; diminished visual acuity; forgetfulness; minor leg cramps': ['hair loss;',
  'depression;',
  'anger issues;',
  'hostility;',
  'visual acuity;',
  'forgetfulness;',
  'leg cramps'],
 'Thinning hair, extreme fatigue, awful joint and bone pain. Most distressing is the loss of the use of my hands. I have developed carpal tunnel syndrome in both hands and I am in alot of pain. Insomnia and hot flashes are also troublesome.': ['Thinning hair,',
  'extreme fatigue,',
  'joint pain.',
  'bone pain.',
  'carpal tunnel syndrome',
  'pain.',
  'Insomnia',
  'hot flashes'],
 'I had been on Tamoxofin for 5 years. I then took Evista for about 1 1 / 2 years when my doctor put me on Femara because it is more effective. I was fine in the beginning. After two years or so, I started having pain in my right hip and down my leg where it was uncomfor

In [222]:
#here we want to create one row per symptom
txt_id = []
text = []
symptom = []
id_t = 0

for key in review_dict:
    
    for sym in review_dict[key]:
        
        txt_id.append(id_t)
        text.append(key)
        res = re.sub(r'[^\w\s]', '', sym)
        symptom.append(res)
    
    id_t += 1

In [223]:
#create a dataframe
d = {'txt_id': txt_id, 'text': text, 'symptom': symptom}
ann_df = pd.DataFrame(data=d)
ann_df.head(30)

,txt_id,text,symptom
0,0,"headaches, dizziness, aching all over",headaches
1,0,"headaches, dizziness, aching all over",dizziness
2,0,"headaches, dizziness, aching all over",aching all over
3,1,Substantial hair loss; depression; anger issue...,hair loss
4,1,Substantial hair loss; depression; anger issue...,depression
5,1,Substantial hair loss; depression; anger issue...,anger issues
6,1,Substantial hair loss; depression; anger issue...,hostility
7,1,Substantial hair loss; depression; anger issue...,visual acuity
8,1,Substantial hair loss; depression; anger issue...,forgetfulness
9,1,Substantial hair loss; depression; anger issue...,leg cramps


In [224]:
#find the position of the side effect in the review
ann_df['start'] = ann_df.apply(lambda x: str(x.text).find(str(x.symptom)), axis=1)
ann_df['length'] = ann_df.apply(lambda x: len(x.symptom), axis=1)
ann_df['end'] = ann_df['start'] + ann_df['length']
ann_df.head(30)

,txt_id,text,symptom,start,length,end
0,0,"headaches, dizziness, aching all over",headaches,0,9,9
1,0,"headaches, dizziness, aching all over",dizziness,11,9,20
2,0,"headaches, dizziness, aching all over",aching all over,22,15,37
3,1,Substantial hair loss; depression; anger issue...,hair loss,12,9,21
4,1,Substantial hair loss; depression; anger issue...,depression,23,10,33
5,1,Substantial hair loss; depression; anger issue...,anger issues,35,12,47
6,1,Substantial hair loss; depression; anger issue...,hostility,49,9,58
7,1,Substantial hair loss; depression; anger issue...,visual acuity,71,13,84
8,1,Substantial hair loss; depression; anger issue...,forgetfulness,86,13,99
9,1,Substantial hair loss; depression; anger issue...,leg cramps,107,10,117


In [226]:
#save as a csv file
ann_df.to_csv(r'/Users/teonaristova/Desktop/Classes/NLP/Project/Standardized_Dataset/labeled_annotated.csv', index=False)

## Creating Datafarme 1 - store text and their ID

In [228]:
#keep only particulat columns
text_df = ann_df[['txt_id', 'text']]
text_df = text_df.drop_duplicates()
text_df.head()

,txt_id,text
0,0,"headaches, dizziness, aching all over"
3,1,Substantial hair loss; depression; anger issue...
10,2,"Thinning hair, extreme fatigue, awful joint an..."
18,3,I had been on Tamoxofin for 5 years. I then to...
23,4,"Aching joints, tiredness, lack of stamina, hea..."


In [229]:
#save as a csv file
text_df.to_csv(r'/Users/teonaristova/Desktop/Classes/NLP/Project/Standardized_Dataset/text_annotated.csv', index=False)

# Combine all datasets into a single dataset

## Datafarme 1 - store text and their ID

In [91]:
#load the CAdec dataset
cadec_df_1 = pd.read_csv(r'/Users/teonaristova/Desktop/Classes/NLP/Project/Standardized_Dataset/text_cadec.csv')
cadec_df_1['dataset'] = 'CADEC'

In [92]:
cadec_df_1.head()

,txt_id,text,dataset
0,LIPITOR.86,"headaches, pain in throat , tingling in side o...",CADEC
1,LIPITOR.92,"Muscle aches and weakness in neck, arms, shoul...",CADEC
2,LIPITOR.952,I have taken Lipitor for 4 years with no probl...,CADEC
3,LIPITOR.946,"Aches and pain from head to toe, very grouchy....",CADEC
4,LIPITOR.45,"Cannot be sure it is the drug, but around the ...",CADEC


In [93]:
cadec_df_1.dtypes

txt_id     object
text       object
dataset    object
dtype: object

In [96]:
#load the PsyTar dataset
psytar = pd.read_csv(r'/Users/teonaristova/Desktop/Classes/NLP/Project/Standardized_Dataset/text_psytar.csv')
psytar['dataset'] = 'PsyTar'
psytar = psytar.rename(columns={"drug_id": "txt_id"})

In [97]:
psytar.head()

,txt_id,text,dataset
0,lexapro.1,"extreme weight gain, short-term memory loss, h...",PsyTar
1,lexapro.2,COMPLETELY DESTROYED SEXUALLY FUNCTIONING - EV...,PsyTar
2,lexapro.3,"Nausea, Blurred Vision, 3 to 5 hours sleep, Su...",PsyTar
3,lexapro.4,"Plenty! First 10 days were HORRIBLE, like a lo...",PsyTar
4,lexapro.5,"Chronic cough, weight gain, no sexual interest...",PsyTar


In [101]:
psytar.dtypes

txt_id     object
text       object
dataset    object
dtype: object

In [103]:
#load the annotated drug review dataset
drug_ann = pd.read_csv(r'/Users/teonaristova/Desktop/Classes/NLP/Project/Standardized_Dataset/text_annotated.csv')
drug_ann['dataset'] = 'Annotated_dataset'
drug_ann['txt_id'] = drug_ann['txt_id'].astype(str)

In [104]:
drug_ann.head()

,txt_id,text,dataset
0,0,"headaches, dizziness, aching all over",Annotated_dataset
1,1,Substantial hair loss; depression; anger issue...,Annotated_dataset
2,2,"Thinning hair, extreme fatigue, awful joint an...",Annotated_dataset
3,3,I had been on Tamoxofin for 5 years. I then to...,Annotated_dataset
4,4,"Aching joints, tiredness, lack of stamina, hea...",Annotated_dataset


In [105]:
drug_ann.dtypes

txt_id     object
text       object
dataset    object
dtype: object

In [110]:
#combine all datasets into a single dataset
pieces = (cadec_df_1,psytar,drug_ann)
combined_df_1 =  pd.concat(pieces, ignore_index = True)
#replace \n and \t from the text
combined_df_1['text'] = combined_df_1.apply(lambda x: str(x.text).replace("\n", " "), axis = 1)
combined_df_1['text'] = combined_df_1.apply(lambda x: str(x.text).replace("\t", " "), axis = 1)

In [111]:
combined_df_1.head(30)

,txt_id,text,dataset
0,LIPITOR.86,"headaches, pain in throat , tingling in side o...",CADEC
1,LIPITOR.92,"Muscle aches and weakness in neck, arms, shoul...",CADEC
2,LIPITOR.952,I have taken Lipitor for 4 years with no probl...,CADEC
3,LIPITOR.946,"Aches and pain from head to toe, very grouchy....",CADEC
4,LIPITOR.45,"Cannot be sure it is the drug, but around the ...",CADEC
5,LIPITOR.775,Muscle and joint pain developed after approx 1...,CADEC
6,LIPITOR.761,aches and pain in both arms and shoulders and ...,CADEC
7,LIPITOR.51,Had muscle pain in my back (primarily small of...,CADEC
8,LIPITOR.79,"Severe B/L shoulder pain, general weakness, ma...",CADEC
9,LIPITOR.991,Brain fog and decreased cognitive skills. Does...,CADEC


In [112]:
#save the combined dataset
text_df.to_csv(r'/Users/teonaristova/Desktop/Classes/NLP/Project/Standardized_Dataset/combined_df_1.csv', index=False)

## Datafarme 2 - text ID, ADR lines, and ADR (long)

In [138]:
#load the cadec dataset
cadec_df_2 = pd.read_csv(r'/Users/teonaristova/Desktop/Classes/NLP/Project/Standardized_Dataset/labeled_cadec.csv')
cadec_df_2['dataframe'] = 'CADEC'
cadec_df_2 = cadec_df_2[['symptom', 'txt_id', 'dataframe', 'start', 'end']]

In [139]:
cadec_df_2.head()

,symptom,txt_id,dataframe,start,end
0,little blurred vision,ARTHROTEC.1,CADEC,29,50
1,feel a bit weird,ARTHROTEC.1,CADEC,437,453
2,gastric problems,ARTHROTEC.1,CADEC,62,78
3,bit drowsy,ARTHROTEC.1,CADEC,9,19
4,Hunger pangs,ARTHROTEC.10,CADEC,0,12


In [119]:
cadec_df_2.dtypes

loc             object
original_ann    object
type            object
symptom         object
start            int64
end              int64
txt_id          object
dataframe       object
dtype: object

In [140]:
#load the psytar dataset
psytar_df_2 = pd.read_csv(r'/Users/teonaristova/Desktop/Classes/NLP/Project/Standardized_Dataset/labeled_psytar.csv')
psytar_df_2['dataframe'] = 'PsyTar'
psytar_df_2 = psytar_df_2[['txt_id', 'symptom', 'start', 'end', 'dataframe']]

In [141]:
psytar_df_2.head()

,txt_id,symptom,start,end,dataframe
0,lexapro.1,extreme weight gain,0,19,PsyTar
1,lexapro.2,COMPLETELY DESTROYED SEXUALLY FUNCTIONING,0,41,PsyTar
2,lexapro.2,completely destroyed my sexual functioning,129,171,PsyTar
3,lexapro.2,PSSD,205,209,PsyTar
4,lexapro.2,PSSD,205,209,PsyTar


In [130]:
psytar_df_2.dtypes

txt_id         object
text           object
symptom        object
start           int64
end             int64
UMLS_code      object
ann_symptom    object
dataframe      object
dtype: object

In [142]:
#load the annotated drug review dataset
drug_ann_df_2 = pd.read_csv(r'/Users/teonaristova/Desktop/Classes/NLP/Project/Standardized_Dataset/labeled_annotated.csv')
drug_ann_df_2['dataframe'] = 'Annotated_dataset'
drug_ann_df_2 = drug_ann_df_2[['txt_id', 'symptom', 'start', 'end', 'dataframe']]

In [143]:
drug_ann_df_2.head()

,txt_id,symptom,start,end,dataframe
0,0,headaches,0,9,Annotated_dataset
1,0,dizziness,11,20,Annotated_dataset
2,0,aching all over,22,37,Annotated_dataset
3,1,hair loss,12,21,Annotated_dataset
4,1,depression,23,33,Annotated_dataset


In [144]:
drug_ann_df_2.dtypes

txt_id        int64
symptom      object
start         int64
end           int64
dataframe    object
dtype: object

In [145]:
#combine all the dataframes
pieces = (cadec_df_2,psytar_df_2,drug_ann_df_2)
combined_df_2 =  pd.concat(pieces, ignore_index = True)

In [146]:
combined_df_2.head(20)

,symptom,txt_id,dataframe,start,end
0,little blurred vision,ARTHROTEC.1,CADEC,29,50
1,feel a bit weird,ARTHROTEC.1,CADEC,437,453
2,gastric problems,ARTHROTEC.1,CADEC,62,78
3,bit drowsy,ARTHROTEC.1,CADEC,9,19
4,Hunger pangs,ARTHROTEC.10,CADEC,0,12
5,canker sores,ARTHROTEC.100,CADEC,107,119
6,headache,ARTHROTEC.100,CADEC,133,141
7,vaginal bleeding,ARTHROTEC.100,CADEC,48,64
8,stomach pain,ARTHROTEC.100,CADEC,93,105
9,cramping,ARTHROTEC.101,CADEC,102,110


In [147]:
#save the combined dataset
combined_df_2.to_csv(r'/Users/teonaristova/Desktop/Classes/NLP/Project/Standardized_Dataset/combined_df_2.csv', index=False)